<h3 style="text-align: center;">Lab 05: Arcpy</h3>

- This lab covers the Arcpy, including topics such as SearchCursor, UpdateCursor and InsertCursor. 

- There are four questions in total. Please provide your code answers directly below each question.

- Make sure to run all cells so that the answers are stored. Once completed, submit the .ipynb file (**ensuring that all answers are included**) to Canvas by **midnight (11:29 PM) on March 10**.

- This lab is worth a total of **80 points** and contributes **8%** toward the final grade.

### 1. SearchCursor (20 points)

Use sedf and groupby() to analyze '2024_Vehicle_Crash', grouping by weather conditions (WEATH_COND) to determine the number of accidents under different weather conditions.

In [5]:
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor

In [6]:
sedf = pd.DataFrame.spatial.from_featureclass("2024_Vehicle_Crash")

sedf.groupby(by = "WEATH_COND").size().reset_index(name="Crash_Count")

WEATH_COND  Crash_Count
0                             Blowing sand, snow/Snow            2
1                                               Clear         3858
2                            Clear/Blowing sand, snow            1
3                                         Clear/Clear          783
4                                        Clear/Cloudy          153
5                              Clear/Fog, smog, smoke            2
6                                         Clear/Other            3
7                                          Clear/Rain           17
8                          Clear/Reported but invalid            2
9                                          Clear/Snow            6
10                                      Clear/Unknown           84
11                                             Cloudy          618
12                                       Cloudy/Clear           22
13                                      Cloudy/Cloudy           55
14                            Cloudy/Fog, smog, smoke            2
15                                        Cloudy/Rain          208
16                           Cloudy/Severe crosswinds            2
17      Cloudy/Sleet, hail (freezing rain or drizzle)            4
18                                        Cloudy/Snow           27
19                                     Cloudy/Unknown            7
20                                   Fog, smog, smoke            6
21                            Fog, smog, smoke/Cloudy            2
22                              Fog, smog, smoke/Rain            3
23                                       Not Reported           60
24                                              Other            3
25                                               Rain          316
26                                         Rain/Clear            7
27                                        Rain/Cloudy           47
28                                          Rain/Rain           72
29                             Rain/Severe crosswinds            1
30        Rain/Sleet, hail (freezing rain or drizzle)           10
31                                          Rain/Snow            5
32             Sleet, hail (freezing rain or drizzle)           13
33  Sleet, hail (freezing rain or drizzle)/Blowing...            2
34      Sleet, hail (freezing rain or drizzle)/Cloudy            5
35        Sleet, hail (freezing rain or drizzle)/Rain            5
36  Sleet, hail (freezing rain or drizzle)/Sleet, ...            2
37        Sleet, hail (freezing rain or drizzle)/Snow           14
38     Sleet, hail (freezing rain or drizzle)/Unknown            2
39                                               Snow          118
40                            Snow/Blowing sand, snow           15
41                                         Snow/Clear            2
42                                        Snow/Cloudy            6
43                                         Snow/Other            2
44                                          Snow/Rain            8
45        Snow/Sleet, hail (freezing rain or drizzle)           60
46                                          Snow/Snow           22
47                                            Unknown          101
48                                    Unknown/Unknown           37

### 2. Insert below new data using InsertCursor (20 points)
- Using InsertCursor to add 'points_data' to the '2024_Vehicle_Crash'

In [ ]:
points_data = [{'CRASH_SEVE': 'Non-fatal injury', 'NUMB_VEHC': 1, 'SHAPE': (-71.791180, 42.249540)},
           {'CRASH_SEVE': 'Unknown', 'NUMB_VEHC': 0, 'SHAPE': (-71.811697, 42.257009)},
           {'CRASH_SEVE': 'Non-fatal injury', 'NUMB_VEHC': 2, 'SHAPE': ( -71.772558, 42.258216)},]

In [7]:
feature_class = "2024_Vehicle_Crash"
fields = ["CRASH_SEVE", "NUMB_VEHC", 'SHAPE'] 

# Data to be inserted
points_data = [{'CRASH_SEVE': 'Non-fatal injury', 'NUMB_VEHC': 1, 'SHAPE': (-71.791180, 42.249540)},
           {'CRASH_SEVE': 'Unknown', 'NUMB_VEHC': 4, 'SHAPE': (-71.811697, 42.257009)},
           {'CRASH_SEVE': 'Non-fatal injury', 'NUMB_VEHC': 2, 'SHAPE': ( -71.772558, 42.258216)},]

# Define the coordinate system for projection (WKID 26986)
spatial_reference = arcpy.SpatialReference(26986)  # NAD83 / UTM Zone 18N

# Use InsertCursor to insert new rows
with arcpy.da.InsertCursor(feature_class, fields) as cursor:
    for point_info in points_data:
        # Create a new point geometry using coordinates from the dictionary
        point = arcpy.Point(point_info['SHAPE'][0], point_info['SHAPE'][1])  # Unpack coordinates
        
        # Print the original X and Y coordinates
        print(f"Original Coordinates: X={point.X}, Y={point.Y}")
        
        # Create a point geometry with the initial spatial reference (WGS 84)
        point_geometry = arcpy.PointGeometry(point, spatial_reference=arcpy.SpatialReference(4326)) 
        
        # Project the point geometry to the desired coordinate system
        projected_geometry = point_geometry.projectAs(spatial_reference)
        
        # Print the projected spatial reference name
        print(f"Projected Spatial Reference: {projected_geometry.spatialReference.name}")
        
        # Extract the projected coordinates for insertion
        pro_geometry_input = (projected_geometry.firstPoint.X, projected_geometry.firstPoint.Y)
        print(f"Projected Coordinates for Insertion: {pro_geometry_input}")
        
        # Insert the row with the specified attributes
        cursor.insertRow((point_info['CRASH_SEVE'], point_info['NUMB_VEHC'], pro_geometry_input))

Original Coordinates: X=-71.79118, Y=42.24954
Projected Spatial Reference: NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001
Projected Coordinates for Insertion: (175970.88872359146, 888827.1929024821)
Original Coordinates: X=-71.811697, Y=42.257009
Projected Spatial Reference: NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001
Projected Coordinates for Insertion: (174280.79875163958, 889662.7849724675)
Original Coordinates: X=-71.772558, Y=42.258216
Projected Spatial Reference: NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001
Projected Coordinates for Insertion: (177510.7100978862, 889785.7905029356)


### 3. Update  "Vehicle_Crash_insert" using UpdateCursor (20 points)
- Add a new field named 'NUM_Class' of type 'Text' to '2024_Vehicle_Crash'.
- Update Values:
    - If 'NUMB_VEHC' is greater than or equal to 3, update the 'NUM_Class' to 'More'.
    - If 'NUMB_VEHC' is less than 3, update 'NUM_Class' to 'Less'.

In [8]:
feature_class = "2024_Vehicle_Crash"

new_field_name = 'NUM_Class'
field_type = 'Text'
arcpy.AddField_management(feature_class, new_field_name, field_type)

fields_update = ['NUMB_VEHC','NUM_Class']

# Create update cursor for feature class 
with arcpy.da.UpdateCursor(feature_class, fields_update) as cursor:
    # For each row, evaluate the WELL_YIELD value (index position 
    # of 0), and update WELL_CLASS (index position of 1)
    for row in cursor:
        if (row[0] >= 3):
            row[1] = 'More'
        else:
            row[1] = "Less"

        # Update the cursor with the updated list
        cursor.updateRow(row)

### 4. Statistics (5 points)

Using the shapefile: 2024_Vehicle_Crash, analyze the distribution of vehicle types involved in crashes.

Each record in the dataset includes a field NUM_Class (created in Task 3), which classifies the type of vehicle involved in the crash into two categories:

"More" and "Less".

Your goal is to:

1. Count the number of crashes involving each type of NUM_Class.
2. Calculate the proportion of each vehicle class relative to the total number of crashes.

You can use arcpy.da.SearchCursor to iterate through the dataset or use the groupby function for spatially enabled DataFrame (sedf) to perform the same analysis.


In [9]:
feature_class = "2024_Vehicle_Crash"

more = 0
less = 0

# Use SearchCursor to iterate through the rows of the feature class
with arcpy.da.SearchCursor(feature_class, ['NUM_Class']) as cursor:
    for row in cursor:
        if row[0] == "More":
            more += 1
        elif row[0] == "Less":
            less += 1

# Calculate total counts
total_count = more + less

# Calculate proportions

more_proportion = more / total_count
less_proportion = less / total_count


# Print the results
print(f"More: {more}, Proportion: {more_proportion:.2%}")
print(f"Less: {less}, Proportion: {less_proportion:.2%}")

More: 1, Proportion: 33.33%
Less: 2, Proportion: 66.67%


In [10]:
sedf = pd.DataFrame.spatial.from_featureclass("2024_Vehicle_Crash")

vehicle_grouped = sedf.groupby("NUM_Class").size().reset_index(name="Crash_Count")

# Calculate proportions
vehicle_grouped["Proportion"] = vehicle_grouped["Crash_Count"] / vehicle_grouped["Crash_Count"].sum()

vehicle_grouped["Proportion"] 

0    0.666667
1    0.333333
Name: Proportion, dtype: float64